## Visualization of Shells

In [1]:
import ipympl
%matplotlib widget
import matplotlib.pyplot as plt
import itkwidgets as itkw
from vtkmodules.vtkIOXdmf2 import vtkXdmfReader
from vtkmodules.vtkCommonDataModel import vtkUnstructuredGrid
from vtkmodules.vtkFiltersGeometry import vtkGeometryFilter
from vtkmodules.vtkCommonExecutionModel import vtkStreamingDemandDrivenPipeline
import ipywidgets as ipw
import numpy as np
import pandas as pd
import h5py as hp
import numba

In [2]:
schedule = pd.read_csv('/home/amit/WorkSpace/UCLA/simulations/PhaseDiagram/Schedule.csv', sep=',')

In [3]:
runw = ipw.Dropdown(value=0, options=[0, 1, 2], description=r'Shell Run')
filew = ipw.IntSlider(min=1, max=600, step=1, value=1, description=r'Simulation', continuous_update=False)
stepsw = ipw.IntSlider(min=0, max=999, step=1, value=0, description=r'Time Step', continuous_update=False)
check0w = ipw.Checkbox(value=True, description='Run0: Red Curve', disabled=False)
check1w = ipw.Checkbox(value=True, description='Run1: Blue Curve', disabled=False)
check2w = ipw.Checkbox(value=True, description='Run2: Purple Curve', disabled=False)
dropdownw = ipw.Dropdown(options=['Volume', 'RMSAngleDeficit'], value='Volume', description='Middle Plot:', disabled=False)
box = ipw.HBox([ipw.VBox([check0w, check1w, check2w]), ipw.VBox([dropdownw, filew]), ipw.VBox([runw, stepsw])])

In [4]:
basedir = '/home/amit/WorkSpace/UCLA/simulations/PhaseDiagram'
xdmffile = '{}/XDMFFiles/Run0/VTKFile-1.xdmf'.format(basedir)
rd = vtkXdmfReader()
rd.SetFileName(xdmffile)
rd.UpdateInformation()
info = rd.GetExecutive().GetOutputInformation(0)
gf = vtkGeometryFilter()
gf.SetInputConnection(rd.GetOutputPort())
gf.Update()
pd = gf.GetOutput()
pd.GetPointData().SetActiveNormals('PointNormals')
vtkview = itkw.view(geometries=pd)

def selectfile(change):
    run = runw.value
    fid = filew.value
    gamma = schedule.Gamma[fid - 1]
    temperature = schedule.Temperature[fid - 1]
    filename = '{}/XDMFFiles/Run{}/VTKFile-{}.xdmf'.format(basedir, run, fid)
    rd.SetFileName(filename)
    rd.UpdateInformation()
    info = rd.GetExecutive().GetOutputInformation(0)
    timesteps = info.Get(vtkStreamingDemandDrivenPipeline.TIME_STEPS())
    stepsw.max = timesteps[-1]
    stepsw.value = timesteps[-1]
    updateview(stepsw.value)
    
    
def updateview(data):
    time = data if isinstance(data, int) else data.new
    info.Set(vtkStreamingDemandDrivenPipeline.UPDATE_TIME_STEP(), time)
    rd.Modified()
    gf.Update()
    pd = gf.GetOutput()
    pd.GetPointData().SetActiveNormals('PointNormals')
    vtkview.geometries = pd

runw.observe(selectfile, names='value')
filew.observe(selectfile, names='value')
stepsw.observe(updateview, names='value')

In [5]:
def getasphericity(r, i):
    """
    Read asphericity from HDF5 file.
    """
    with hp.File('../RawData/Run{0}/VTKFile-{1}.h5'.format(r, i), 'r') as hfile:
        asph = hfile['Asphericity'][:]
    return asph


def get_msd_etc(rid, fid, etc):
    filename = '../RawData/Run{}/DetailedOutput-{}.h5'.format(rid, fid)
    with hp.File(filename, 'r') as hfile:
        etcarr = hfile[etc][::2000]
        msd = hfile['MSD'][::2000]
    return etcarr, msd

In [6]:
plt.ioff()
plt.clf()
plt.style.use('ggplot')
plt.rc('text', usetex=True)

fig, (ax0, ax1, ax2) = plt.subplots(3, 1, sharex=True)
fig.set_tight_layout(True)
fig.canvas.layout.max_width='750px'
fig.canvas.layout.height='700px'

ax0.set_ylabel('MSD')
ax1.set_ylabel('Volume')
ax2.set_ylabel('Asphericity')
ax0.set_ylim(0.0, 5.0)
ax1.set_ylim(20.0, 50.0)
ax2.set_ylim(-0.005, 0.05)

ax2.set_xlabel(r'Steps')
_ = ax2.set_xlim(1, 2000000)

In [7]:
time = np.arange(0, 2000000, 2000)

text = r'$\gamma = {0:9.3f}$, $1/\beta = {1:6.4f}$'.format(
    schedule.Gamma[0], schedule.Temperature[0])
title = ax0.set_title(text)

etcarr0, msdarr0 = get_msd_etc(0, 1, 'Volume')
etcarr1, msdarr1 = get_msd_etc(1, 1, 'Volume')
etcarr2, msdarr2 = get_msd_etc(2, 1, 'Volume')

msd0, = ax0.plot(time, msdarr0)
msd1, = ax0.plot(time, msdarr1)
msd2, = ax0.plot(time, msdarr2)
msdl  = [msd0, msd1, msd2]

etc0, = ax1.plot(time, etcarr0)
etc1, = ax1.plot(time, etcarr1)
etc2, = ax1.plot(time, etcarr2)
etcl  = [etc0, etc1, etc2]

asph0, = ax2.plot(time, getasphericity(0, 1))
asph1, = ax2.plot(time, getasphericity(1, 1))
asph2, = ax2.plot(time, getasphericity(2, 1))
aspl = [asph0, asph1, asph2]

tl0 = ax0.axvline(x=0, ls='--', lw=0.5, c='k')
tl1 = ax1.axvline(x=0, ls='--', lw=0.5, c='k')
tl2 = ax2.axvline(x=0, ls='--', lw=0.5, c='k')
tvlines = [tl0, tl1, tl2]

In [8]:
ta = ipw.Textarea(value='')
display(ta)

Textarea(value='')

In [9]:
def updateasphplot(change):
    ta.value = 'updateasphplot() called'
    fid = filew.value
    c0 = check0w.value
    c1 = check1w.value
    c2 = check2w.value
    etc= dropdownw.value
    
    gamma = schedule.Gamma[fid - 1]
    temperature = schedule.Temperature[fid - 1]
    
    text = r'$\gamma = {0:9.3f}$, $1/\beta = {1:6.4f}$'.format(gamma, temperature)
    title.set_text(text)
    
    for i, c in enumerate((c0, c1, c2)):
        if c:
            asph = getasphericity(i, fid)
            aspl[i].set_data(time, asph)
            etcarr, msd = get_msd_etc(i, fid, etc)
            etcl[i].set_data(time, etcarr)
            msdl[i].set_data(time, msd)
        else:
            msdl[i].set_data([], [])
            etcl[i].set_data([], [])
            aspl[i].set_data([], [])
            
    tvalue = stepsw.value * 2000
    for tl in tvlines:
        tl.set_xdata(np.array([tvalue, tvalue]))
        
    fig.canvas.draw()
    fig.flush_events()


def updatetimelines(change):
    tvalue = stepsw.value * 2000
    for tl in tvlines:
        tl.set_xdata(np.array([tvalue, tvalue]))
    fig.canvas.draw()
    fig.flush_events()

    
def updatemiddleplot(change):
    etcvalue = change.new
    ax1.set_ylabel(etcvalue)
    if etcvalue is 'RMSAngleDeficit':
        ax1.set_ylim(0.0, 1.4)
    elif etcvalue is 'Volume':
        ax1.set_ylim(20.0, 50.0)
    updateasphplot(change)
 

dropdownw.observe(updatemiddleplot, names='value')
stepsw.observe(updatetimelines, names='value')

stepsw.observe(updateasphplot, names='value')
filew.observe(updateasphplot, names='value')
check0w.observe(updateasphplot, names='value')
check1w.observe(updateasphplot, names='value')
check2w.observe(updateasphplot, names='value')

In [10]:
display(ipw.VBox([box, ipw.HBox([fig.canvas, vtkview])]))